In [1]:
import cv2

import ezkl
import os
import json

import tf2onnx
import onnx

import torch
from onnx2torch import convert

import sys
import numpy
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.backends.cudnn as cudnn
from collections import OrderedDict
from deepface import DeepFace

In [2]:
from facenet_pytorch import MTCNN
from PIL import Image

In [4]:
mtcnn = MTCNN(keep_all=False)

In [5]:
def ReadImage(image_path):
    # Read the image using OpenCV
    img = cv2.imread(image_path)
    
    # Convert the image from BGR to RGB
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Convert to PIL Image format
    img_pil = Image.fromarray(img_rgb)
    # img_pil.show()
    
    # Detect and align the face
    aligned_face, prob = mtcnn(img_pil, return_prob=True)

    if aligned_face is not None:
        # Convert the tensor to a numpy array and change from CHW to HWC format
        print(aligned_face.shape)

        aligned_face_np = aligned_face.permute(1, 2, 0).cpu().numpy()

        # plt.imshow(aligned_face_np)

        aligned_face_np = numpy.clip(aligned_face_np, 0, 1) * 255

        aligned_face_np = aligned_face_np.astype(numpy.uint8)

        # aligned_face_np = aligned_face.permute(1, 2, 0).mul(255).byte().numpy()

        aligned_face_pil = Image.fromarray(aligned_face_np)
        
        # Display the PIL Image
        # aligned_face_pil.show(title=f"Aligned Face")
        
        # Convert the numpy array to BGR format expected by OpenCV
        aligned_face_bgr = cv2.cvtColor(aligned_face_np, cv2.COLOR_RGB2BGR)
        
        # Resize the image to 96x96 pixels (OpenFace input size)
        resized_face = cv2.resize(aligned_face_bgr, (96, 96))

        # cv2.imshow(f'Aligned Face', resized_face)
        
        # Normalize the pixel values
        resized_face = resized_face.astype(numpy.float32) / 255.0
        
        # Convert to PyTorch tensor and change from HWC to CHW format
        aligned_face_tensor = torch.tensor(resized_face).permute(2, 0, 1).float()
        
        return aligned_face_tensor.unsqueeze(0)  # Add batch dimension
    else:
        print(f"No face detected in {image_path}")
        return None


img_paths = ['emmapassport.png']
imgs = []
for img_path in img_paths:
    imgs.append(ReadImage(img_path))

I_ = torch.cat(imgs, 0)
I_ = Variable(I_, requires_grad=False)

torch.Size([3, 160, 160])


In [6]:
data_array = (I_.detach().numpy()).reshape([-1]).tolist()
data = dict(input_data = [data_array])
data_path = os.path.join('input.json')
settings_path = os.path.join('settings.json')
model_path = os.path.join('openface_pt_native.onnx')

In [4]:
# Serialize data into file:
json.dump( data, open(data_path, 'w' ))

py_run_args = ezkl.PyRunArgs()
py_run_args.input_visibility = "private"
py_run_args.output_visibility = "public"
py_run_args.param_visibility = "fixed" # private by default

# What is the settings file?
res = ezkl.gen_settings(model_path, settings_path, py_run_args=py_run_args)

assert res == True

In [8]:
cal_path = os.path.join("calibration.json")

In [10]:

img_paths = ['florentpassport.jpeg', "kayleepassport.jpg", "emmapassport2.png", "florentselfie.jpg", "emmaselfie.png"]
imgs = []
for img_path in img_paths:
    imgs.append(ReadImage(img_path))

I_ = torch.cat(imgs, 0)
I_ = Variable(I_, requires_grad=False)

data_array = (I_).reshape([-1]).tolist()
data = dict(input_data = [data_array])

json.dump(data, open(cal_path, 'w'))

(3, 96, 96)
(3, 96, 96)
(3, 96, 96)
(3, 96, 96)
(3, 96, 96)


In [ ]:
res = await ezkl.calibrate_settings(cal_path, model_path, settings_path, target="resources", scales=[13], scale_rebase_multiplier=[1], only_range_check_rebase=True)

In [8]:
compiled_model_path = os.path.join('network.compiled')

In [9]:
res = ezkl.compile_circuit(model_path, compiled_model_path, settings_path)
assert res == True

In [7]:
res = await ezkl.get_srs( settings_path)

In [ ]:
# now generate the witness file 
witness_path = os.path.join('witness.json')
pk_path = os.path.join('test.pk')
vk_path = os.path.join('test.vk')

res = await ezkl.gen_witness(data_path, compiled_model_path, witness_path)
assert os.path.isfile(witness_path)

In [ ]:
# HERE WE SETUP THE CIRCUIT PARAMS
# WE GOT KEYS
# WE GOT CIRCUIT PARAMETERS
# EVERYTHING ANYONE HAS EVER NEEDED FOR ZK



res = ezkl.setup(
        compiled_model_path,
        vk_path,
        pk_path,
        
    )

assert res == True
assert os.path.isfile(vk_path)
assert os.path.isfile(pk_path)
assert os.path.isfile(settings_path)

PanicException: dynamic lookup or shuffle should only have one block

calibration failed max lookup input (-4202994348144, 118871353118596) is too large
Using 5 columns for non-linearity table.
Using 5 columns for non-linearity table.
Using 5 columns for non-linearity table.
Using 5 columns for non-linearity table.
Using 5 columns for non-linearity table.
Using 5 columns for non-linearity table.
Using 9 columns for non-linearity table.
Using 9 columns for non-linearity table.
Using 9 columns for non-linearity table.
Using 9 columns for non-linearity table.
Using 9 columns for non-linearity table.
Using 9 columns for non-linearity table.
calibration failed extended k is too large to accommodate the quotient polynomial with logrows 20
circuit creation from run args failed: SigBitTruncationError
circuit creation from run args failed: SigBitTruncationError
calibration failed max lookup input (-78968271, 489000960) is too large
calibration failed max lookup input (-67320924713427, 1897707960036100) is too large
calibration failed max lookup input (-72087465, 